NAME= SAKSHI

PRN-23070243039

# INSTALLING LIBRARIES

In [ ]:
!pip install rdflib chromadb sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.1/565.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 15.0 MB/s eta 0

# IMPORTING THE LIBRARIES


In [9]:
import chromadb
from rdflib import Graph, Namespace, RDF
from sentence_transformers import SentenceTransformer


# CREATING GRAPH

In [10]:

# creating graph
graph=Graph()
exp=Namespace("http://test.org/")


#adding triples
graph.add((exp.Alice,RDF.type, exp.Person))
graph.add((exp.Alice,exp.knows, exp.Bob))
graph.add((exp.Bob,exp.worksAt, exp.AcmeCorp))

#converting triples to simple text
sentences=[]
for subj,pred,obj in graph:
              s=subj.split("#")[-1] if "#" in str(subj) else subj.split("/")[-1]
              p=pred.split("#")[-1] if "#" in str(pred) else pred.split("/")[-1]
              o=obj.split("#")[-1] if "#" in str(obj) else obj.split("/")[-1]
              sentences.append(f"{s} {p} {o}")



# EMBEDDING THE SENTENCES AND STORING IN CHROMADB

In [12]:
# embedding the sentences
model=SentenceTransformer('all-MiniLM-L6-v2')
embeddings=model.encode(sentences)

# making collection
client=chromadb.Client()
collection=client.create_collection(name="my_collection1")

#storing in chromad
doc_ids = []
for i in range(len(sentences)):
    doc_ids.append(str(i))

for i in range(len(sentences)):
    print(f"ID: {doc_ids[i]} | Sentence: {sentences[i]}")

# Adding documents (text), embeddings (vectors) and ids to the Chromadb
collection.add(
    documents=sentences,
    embeddings=embeddings,
    ids=doc_ids
)

# asking queries
query = input("ASK ANYTHING")
query_embedding = model.encode([query])[0]

#printing result
result = collection.query(query_embeddings=[query_embedding], n_results=1)
print("RESULT:", result['documents'][0][0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

ID: 0 | Sentence: Alice knows Bob
ID: 1 | Sentence: Bob worksAt AcmeCorp
ID: 2 | Sentence: Alice type Person
ASK ANYTHINGWhere does Bob work?
RESULT: Bob worksAt AcmeCorp
